<a href="https://colab.research.google.com/github/plus2net/Python-basics/blob/main/pytorch_basic_iris_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the Iris dataset
iris = load_iris()
X = iris.data  # shape (150, 4)
y = iris.target  # shape (150,)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_ds = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)

# Define the neural network
class IrisNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 3)  # 3 classes
        )
    def forward(self, x):
        return self.net(x)

model = IrisNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for xb, yb in train_loader:
        preds = model(xb)
        loss = criterion(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).float().mean().item()
    print(f"Test Accuracy: {accuracy:.4f}")

torch.save(model.state_dict(),'smo.pt') # creating a Model file and saving

Epoch 1/30, Loss: 1.0219
Epoch 2/30, Loss: 0.7556
Epoch 3/30, Loss: 0.4410
Epoch 4/30, Loss: 0.2688
Epoch 5/30, Loss: 0.1926
Epoch 6/30, Loss: 0.1381
Epoch 7/30, Loss: 0.1011
Epoch 8/30, Loss: 0.0942
Epoch 9/30, Loss: 0.0828
Epoch 10/30, Loss: 0.0574
Epoch 11/30, Loss: 0.0564
Epoch 12/30, Loss: 0.0582
Epoch 13/30, Loss: 0.0473
Epoch 14/30, Loss: 0.0550
Epoch 15/30, Loss: 0.0431
Epoch 16/30, Loss: 0.0403
Epoch 17/30, Loss: 0.0486
Epoch 18/30, Loss: 0.0436
Epoch 19/30, Loss: 0.0486
Epoch 20/30, Loss: 0.0365
Epoch 21/30, Loss: 0.0365
Epoch 22/30, Loss: 0.0348
Epoch 23/30, Loss: 0.0354
Epoch 24/30, Loss: 0.0341
Epoch 25/30, Loss: 0.0377
Epoch 26/30, Loss: 0.0599
Epoch 27/30, Loss: 0.0673
Epoch 28/30, Loss: 0.0410
Epoch 29/30, Loss: 0.0336
Epoch 30/30, Loss: 0.0435
Test Accuracy: 0.9667


Using the above created model ( last line )  file to load the pytorch model. Without this file also we can pass the model to below code.

In [ ]:
model=IrisNN()
model.load_state_dict(torch.load('smo.pt'))

<All keys matched successfully>

In [ ]:
import torch
import numpy as np

# Example raw input: [sepal length, sepal width, petal length, petal width]
# e.g., a flower with measurements 5.1, 3.5, 1.4, 0.2
raw_sample = np.array([[5.1, 3.5, 1.4, 0.2]])

# Scale using the previously fitted StandardScaler
sample_scaled = scaler.transform(raw_sample)

# Convert to PyTorch tensor
sample_tensor = torch.tensor(sample_scaled, dtype=torch.float32)

# Perform inference
model.eval()
with torch.no_grad():
    logits = model(sample_tensor)
    probs = torch.softmax(logits, dim=1)
    pred_class = torch.argmax(probs, dim=1).item()
    pred_prob = probs[0, pred_class].item()

# Map numeric prediction to species name
species = iris.target_names[pred_class]

print(f"Predicted species: {species} (class {pred_class})")
print(f"Confidence: {pred_prob:.4f}")


Predicted species: setosa (class 0)
Confidence: 1.0000
